In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import PIL.Image as Image
import os

In [2]:
classes = [
    'Bird Drop',
    'Clean',
    'Dusty',
    'Electrical Damage',
    'Physical Damage',
    'Snow-covered'
]

In [3]:
model = torch.load('Best-Model.pth')

In [4]:
mean = [0.4243, 0.4522, 0.5050]
std = [0.2150, 0.2035, 0.1913]

image_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
])

In [5]:
def classify_single_image(model, image_transforms, image_path, classes):
    model = model.eval()
    image = Image.open(image_path)
    image = image_transforms(image).float()
    image = image.unsqueeze(0)

    output = model(image)
    _, prediction = torch.max(output.data, 1)

    print(classes[prediction.item()])

In [6]:
classify_single_image(model, image_transforms,'./Single Images/Clean.jpg', classes)

Clean


In [7]:
classify_single_image(model, image_transforms,'./Single Images/Damaged.jpg', classes)

Bird Drop


In [8]:
classify_single_image(model, image_transforms,'./Single Images/Dusty.png', classes)

Dusty
